In [2]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, date
import random

## CFM 101: Group Assignment - Python Roboadvisor
### Team Number: 15
### Team Member Names: Landon Trinh, Ethan Zemelman, Jessie Deng
### Team Strategy Chosen: SAFE

## Goal
- We have decided to focus on dynamically building the safest portfolio given a file of unknown stock tickers
- We recognize that the roboadvisor will run from November 25, 2023 to December
4, 2023
- The deviation will be calculated by taking calculating the difference between the final value of the portfolio and the initial portfolio value ($750,000)
- Ultimately, our aim is to have our portfolio deviate in nominal value. In other words, in the end, we want our portfolio value to have changed as little as possible


## Introduction
> "Theory will only take you so far." - J. Robert Oppenheimer

In theory, our trading strategy should produce a portfolio that deviates in nominal value the least by taking into consideration the following:
- Beta
- Diversification
- Correlation
- Standard deviation
- Expected returns

Our goal will be to tell a convincing story to WHY we are picking our stocks. We will calculate and discuss statistics, display and intepret graphs, and explain our thought process

## 1. Setup
Before implementing our trading strategy, we will initialize required and useful constants as part of the rules:
- Currency of valid stocks (USD or CAD)
- Required average monthly volume (150,000 shares)
- The number of stocks we wish to purchase on the start date (10-22 stocks)
- Time interval (Janurary 1, 2023 - October 31, 2023)
- Minimum number of trading days for month (18 days)
- Minimum stock weighting: $\frac{100}{2n}$%, $n$ = number of stocks in portfolio
- Maximum stock weighting: 20%
- Initial investment amount: $750,000 CAD
- Buying date of roboadvisor: November 25, 2023 - December 4, 2023
- Trading fee for each stock trade: $4.95 CAD

In the end, our roboadvisor should create two DataFrames:

1. $\verb|Portfolio_Final|\\$
- Index: Starts at 1 and ends at number of stocks in portfolio
- Headings: Ticker, Price (price of stock on Nov 25), Currency (CAD or USD), Shares, Value, Weight (adds to 100%)

2. $\verb|Stocks_Final|\\$
We should output this DataFrame to a CSV file titled "Stocks_Group_15.csv"
- Index: Same as "Portfolio Final"
- Headings: Tickers and Shares from "Portfolio_Final"


In [1]:
# Investment amount (CAD)
capital = 750_000

# Number of stocks to buy for portfolio
num_stocks = 15

# Maximum and minimum weightings of each stock in portfolio
min_weight = 1 / (2 * num_stocks)
max_weight = 0.20

# Start and end date for roboadvisor
# start_date = "2023-11-25"
# end_date = "2023-12-04"

# Filtering requirements
min_trading_days = 18
min_avg_volume = 150000

## 2. Filtering
After reading in the CSV file containg stock tickers, we must filter the list of stocks to make sure they are valid stock tickers according to the following rules:
- Include stocks that have an average monthly volume of at leaest 150,000 shares based on Jan 1, 2023 - Oct 31, 2023 (drop any months that don't have at least 18 trading days)
- Stock denominated in USD or CAD

In [3]:
# Read in CSV ticker file
tickers = pd.read_csv("tickers_example.csv", header=None)
tickers = tickers.rename(columns={0: "ticker"})
tickers_lst = tickers["ticker"].tolist()
tickers.head()

,ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


In [4]:
# Set parameters for filtering tickers
filter_start_date = "2023-01-01"
filter_end_date = "2023-10-31"
filter_interval = "1mo"

In [5]:
# Determines months with less than 18 trading days
def get_short_months(market_index):
    short_months = []
    for month in range(1, 11):
        trading_days = len(market_index.history(start=str(date(2023, month, 1)), end=str(date(2023, month+1, 1))))
        if trading_days < min_trading_days:
            short_months.append(month)
    return short_months

# Keeps stocks with valid average monthly volume
def filter_volume(tickers, short_months):

    # Retrieve monthly volume data for tickers
    volume_data = yf.download(tickers=tickers, interval="1mo", start=filter_start_date, end=filter_end_date).Volume

    # Drop short months from volume DataFrame
    for short_month in short_months:
        volume_data.drop(str(date(2023, short_month, 1)))

    # Determine whether stocks meets average monthly volume requirement
    for ticker in tickers:
        if (volume_data[ticker]).mean() < min_avg_volume:
            print(f"{ticker} does not meet the required minimum average monthly volume")
            tickers.remove(ticker)

    # Return finalized list of tickers
    return tickers


# Retrieve filtered tickers
def filter_tickers(tickers):
    
    # Initialize list to separately store CAD and USD tickers
    cad_tickers = []
    usd_tickers = []
    
    for ticker in tickers:
        try:
            stock_ticker = yf.Ticker(ticker)
            base_currency = stock_ticker.fast_info["currency"]
            
            # Store ticker in appropriate list
            if base_currency == "CAD":
                cad_tickers.append(ticker)
            
            elif base_currency == "USD":
                usd_tickers.append(ticker)
    
        except:
            print(f"{ticker} may be delisted")

    # Determine months that have less than 18 trading days for CAD and USD stocks
    cad_short_months = get_short_months(yf.Ticker("^GSPTSE"))
    usd_short_months = get_short_months(yf.Ticker("^GSPC"))

    # Filter months that have an average monty volume of less than 150k
    filtered_cad_tickers = filter_volume(cad_tickers, cad_short_months)
    filtered_usd_tickers = filter_volume(usd_tickers, usd_short_months)

    # Add to single list
    filtered_tickers = filtered_cad_tickers + filtered_usd_tickers
    return filtered_tickers
    
filtered_tickers = filter_tickers(tickers_lst)
print(filtered_tickers)

AGN may be delisted
CELG may be delisted
MON may be delisted
RTN may be delisted
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  32 of 32 completed
['RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


In [43]:
# Download stock data
stock_data = yf.download(tickers=filtered_tickers, interval="1d", start=filter_start_date, end=filter_end_date).Close.dropna()

# Get daily returns of stocks
daily_returns = stock_data.pct_change().dropna()
daily_returns.index = daily_returns.index.tz_localize('UTC')
daily_returns.index = daily_returns.index.tz_convert('UTC')
daily_returns.head()

[*********************100%%**********************]  36 of 36 completed


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-04 00:00:00+00:00,0.010314,0.008067,0.014875,-0.003404,0.014778,-0.007924,0.023246,0.042223,0.018800,-0.006676,...,0.040392,0.008904,0.037303,0.012158,0.013003,0.036517,-0.027264,0.007997,0.010440,0.031138
2023-01-05 00:00:00+00:00,-0.010605,-0.001222,-0.003687,-0.023613,-0.005481,-0.023726,-0.027302,0.006629,-0.002050,0.002880,...,-0.019098,-0.002942,-0.035171,-0.001126,-0.027024,-0.013241,-0.028821,-0.029440,-0.018463,-0.007821
2023-01-06 00:00:00+00:00,0.036794,0.018717,0.013809,0.023690,0.016375,0.035611,0.025541,0.039075,0.009979,0.028204,...,0.054296,0.012734,0.014950,0.015408,-0.000463,0.049302,0.000082,0.043973,0.029395,0.014014
2023-01-09 00:00:00+00:00,0.004089,-0.029361,-0.001602,0.016864,-0.010535,0.014870,0.001532,-0.020798,-0.015112,-0.016222,...,-0.006329,0.007437,0.005044,-0.002221,-0.003126,0.008678,0.000122,-0.002594,0.015312,0.006478
2023-01-10 00:00:00+00:00,0.004456,-0.012495,0.015158,0.004310,-0.006576,0.028732,0.010638,-0.009014,0.006787,0.024534,...,0.020068,0.011416,-0.016061,0.001113,-0.001161,0.009735,-0.008285,0.003878,-0.016071,0.001073


## 3. Stock Analysis
In this step we will choose our stocks based on various measures.
- Get standard deviation of each stock
- Get the beta of each stock
- Get the expected returns of each stock
- Create a correlation matrix of all the stocks

## 3.1 - Standard Deviation

Standard deviation is a statistic that measures the dispersion of a dataset relative to its mean.

In finance, standard deviation is often used as a measure or a relative riskiness of an asset. For example, a volatile stock will have a high standard deviation while a stable blue-chip stock will have a lower standard deviation.

To create a safe portfolio, we want to select stocks that are less volatile. Hence, we will take a look at stocks that have the lowest standard deviations.

The function below simplies calculates the standard deviation of the daily historical returns of each stocks, and sorts it from lowest to highest.

$$\sigma_X=\sqrt{\frac{\sum(x_i-\overline{X})^2}{N}}
$$

In [9]:
# Display stocks with lowest standard deviations
def get_stock_std(daily_returns):
    std_returns = pd.DataFrame(daily_returns.std())
    std_returns = std_returns.rename(columns={0: "std"})
    std_returns = std_returns.sort_values(by="std")
    return std_returns

stock_std = get_stock_std(daily_returns)
stock_std

,std
KO,0.008796
RY.TO,0.009367
PG,0.009581
PEP,0.009822
CL,0.010570
T.TO,0.010654
TD.TO,0.010948
PM,0.011099
BMY,0.011738
MO,0.011773


## 3.2 Beta
The beta value of a stock measures its volatility relative to the broader market. A beta value greater than 1 indicates that the stock is more volatile than the market and a beta value less than 1 indicates a more stable stock. Since we are choosing to go safe with our portfolio, stocks need to be chosen so that the beta values are as low as possible.

We will be using the S&P 500 as out market index. Since a volatile stock may lead to capital gains/losses, we will seek the lowest magnitude beta values.

The aim will be to use Beta as a statistical measure to quantify our various portfolios which will allow us to choose the appropirate set of stocks.
\begin{align*}
\beta=\frac{Cov(r_i,r_m)}{Var(r_m)}
\end{align*}

In [57]:
# Set S&P 500 as market index
market_index = yf.download(tickers="^GSPC", start=filter_start_date, end=filter_end_date)["Close"]
market_returns = market_index.pct_change().dropna()
market_returns.index = market_returns.index.tz_localize('UTC')
market_returns.index = market_returns.index.tz_convert('UTC')
market_variance = market_returns.var()

def get_stock_betas(stocks): 
        betas = {}
        for stock in stocks:
                stock_returns = daily_returns[stock]
                beta = stock_returns.cov(market_returns) / market_variance
                betas[stock] = beta
        
        # Return sorted betas in increasing order
        return dict(sorted(betas.items(), key=lambda item: item[1]))

betas = get_stock_betas(filtered_tickers).items()

betas_df = pd.DataFrame(betas, columns=["ticker", "beta"])
betas_df

[*********************100%%**********************]  1 of 1 completed


,ticker,beta
0,ABBV,0.159620
1,MRK,0.233888
2,UNH,0.282467
3,CL,0.295556
4,LMT,0.310787
5,PEP,0.354425
6,KO,0.360745
7,BMY,0.386441
8,T.TO,0.394628
9,PG,0.411532


## 3.3 - Correlation

In finance, correlation is a statistic that measures the dgree to which two securities move in relation to each other.
Correlations are computed as the correlation coefficent which has a value between -1 and 1.

$$ COR(X,Y)=\frac{COV(X,Y)}{\sigma_X \times \sigma_Y} $$

In this case, we will measure the correlation of each stock with the S&P 500 has the benchmark index, where

$\sigma_X$ = deviation of the stock


$\sigma_Y$ = deviation of the market index

In [13]:
# Get market index data
market_index = "^GSPTSE"
market_return = pd.DataFrame(yf.download(tickers=market_index, interval="1d", start=filter_start_date, end=filter_end_date).Close.pct_change())
market_return = market_return.rename(columns={"Close": "S&P500"})

# Combine into single DataFrame
stock_market_data = pd.concat([stock_data, market_return], axis=1).dropna()
stock_market_returns = stock_market_data.pct_change()
stock_market_returns

# Get correlation of each stock with the S&P 500
correlation_index = pd.DataFrame(stock_market_returns.corr()["S&P500"])[:-1]
correlation_index = correlation_index.rename(columns={"S&P500": "corr"})

[*********************100%%**********************]  1 of 1 completed


In [75]:
# Get stocks with highest correlation with S&P 500
lowest_correlations = correlation_index.nsmallest(10, "corr")
lowest_correlations

,corr
AMZN,-0.154492
ACN,-0.112333
TD.TO,-0.103305
AAPL,-0.095170
LLY,-0.091421
PEP,-0.073338
RY.TO,-0.067776
CL,-0.062437
SHOP.TO,-0.060543
BA,-0.059102


In [77]:
# Get stocks with lowest correlation with S&P 500
highest_correlations = correlation_index.nlargest(10, "corr")
highest_correlations

,corr
BMY,0.079121
PFE,0.069572
AXP,0.057096
MRK,0.039944
BK,0.024997
MO,0.022872
UNH,0.019315
UNP,0.004768
BIIB,0.001542
BAC,-0.001682


In [7]:
std_dict = {}
betas = {}
exp_returns = {}

final_closings = pd.DataFrame()
Stocks_Final = pd.DataFrame()

## 4. Portfolio Optimization
- Create random weights for each stock, and create n number of random portfolios
- Choose the portfolio with the lowest expected returns

In [8]:
Portfolio_Final = pd.DataFrame()

# TODO: fix random_weights function (currently doesn't obey the max_val)
def random_weights(n, min_val, max_val):
    """
    Generates a list of n number of random weights, each between min and max, that sum to 1
    """

    # Adjust the weights so they sum up to 1 with specified constraints
    # by initially giving each weight the minimum value
    remaining = 1 - n * min_val
    weights = [min_val] * n
    
    for i in range(n - 1):
        # Calculate the maximum value the next weight can take
        max_next = min(remaining, max_val - min_val)
        # Generate a random weight between min_val and max_next
        weight = min_val + random.uniform(0, max_next)
        remaining -= (weight - min_val)
        
        if remaining < 0:
            # This error will occur if the min is too high and n isn't small enough
            raise ValueError("Not enough value left to distribute among the remaining weights.")
        
        weights[i] += (weight - min_val)

    # Allocate the remaining value to the last weight
    weights[-1] += remaining

    return weights

def random_portfolios(num_portfolios, closing_prices):
    """
    Generates a list of num_portfolios number of random portfolios (each stored in a dataframe) by randomly assigning weights to each stock

    Parameters:
    num_portfolios (int): Number of random portfolios to generate
    closing_prices (pd.DataFrame): Dataframe containing closing prices for each stock

    Returns:
    portfolios (dictionary): Dictionary containing the randomly generated portfolios.
                       Each portfolio is a dataframe containing the stocks' daily values in the portfolio based on their weights.
    expected_returns (dictionary): Dictionary containing the expected returns for each portfolio
    """

    # Remove NaN values from closing prices or we may experience some issues
    closing_prices.dropna(inplace=True)

    portfolios = {}
    expected_returns = {}
    weightings = {}

    # Create the random portfolios, each containing the stocks' daily values based on their weights
    for i in range(num_portfolios):
        weights = random_weights(closing_prices.shape[1], min_weight, max_weight)
        weightings[i] = weights

        investment_per_stock = weights * capital
        # Calculate how many shares to buy (based on the closing price of the first day)
        num_shares = investment_per_stock / closing_prices.iloc[0]

        # Calculate the daily value of each stock in the portfolio
        portfolio = closing_prices * num_shares
        portfolios[i] = portfolio

        # Calculate the expected return of the portfolio
        # Each row in this dataframe is the total value of the portfolio on that day
        total_portfolio_value = portfolio.sum(axis=1)
        # Calculate the returns of the portfolio
        returns = total_portfolio_value.pct_change()

        # TODO: double check expected return calculation
        expected_return = returns.mean()
        expected_returns[i] = expected_return
    
    return portfolios, expected_returns, weightings

In [9]:
num_portfolios = 1000 # Number of random portfolios to generate

# TODO: remove the variable below after testing
final_closings = stock_data.iloc[:, :10] # Get first 10 stocks just for testing

rand_portfolios, expected_returns, weightings = random_portfolios(num_portfolios, final_closings)

# Pick the portfolio with the expected return closest to zero
# Here, we just use the min function to find the smallest absolute value in the dictionary
optimal_portfolio = min(expected_returns, key=lambda x: abs(expected_returns[x]))
print(f"The optimal portfolio has an expected return of about {float(expected_returns[optimal_portfolio]):.15%}.")

optimal_weights_df = pd.DataFrame(weightings[optimal_portfolio], index=final_closings.columns, columns=["Weighting"])
print(f"\nHere are the best weights for each stock:")
display(optimal_weights_df)

print("The optimal portfolio is:")
display(rand_portfolios[optimal_portfolio])

The optimal portfolio has an expected return of about 0.000007864015579%.

Here are the best weights for each stock:


,Weighting
AAPL,0.079518
ABBV,0.067201
ABT,0.068711
ACN,0.045215
AIG,0.082876
AMZN,0.099093
AXP,0.122478
BA,0.079784
BAC,0.192359
BIIB,0.162765


The optimal portfolio is:


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB
Date,,,,,,,,,,
2023-01-03,59638.379046,50400.718808,51533.072768,33911.129833,62156.719348,74319.969163,91858.775730,59838.060146,144269.281893,122073.893265
2023-01-04,60253.503087,50807.325636,52299.624746,33795.690280,63075.291852,73731.090092,93994.154028,62364.617276,146981.600790,121258.959560
2023-01-05,59614.535591,50745.249089,52106.812599,32997.665115,62729.590375,71981.776543,91427.952037,62778.055767,146680.233849,121608.214910
2023-01-06,61807.998877,51695.033988,52826.337486,33779.377795,63756.815575,74545.130557,93763.136494,65231.111371,148144.023175,125038.088558
2023-01-09,62060.723497,50177.239452,52741.687288,34349.039652,63085.167314,75653.607254,93906.741215,63874.428997,145905.285592,123009.718322
...,...,...,...,...,...,...,...,...,...,...
2023-10-24,82703.131307,45412.789901,44587.063531,37152.171780,59134.319131,111332.732325,90172.951760,55847.631500,109654.989400,112885.774692
2023-10-25,81587.327446,45086.882111,44003.919717,36724.298352,60210.925412,105123.526491,89611.013613,54429.695518,109999.409935,110472.327168
2023-10-26,79579.827831,45068.259620,44196.735452,36643.995739,60102.276484,103547.409948,89498.620267,54846.194820,112453.415484,107946.938652


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.